# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [9]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [10]:
#check missing values from previous question

#deal with random missing values
df['Air Temperature'] = df['Air Temperature'].ffill()
df['Barometric Pressure'] = df['Barometric Pressure'].ffill()


In [11]:
#deal with patterned missing values

# Missing values for Wet Bulp Temp, Total Rain, Precipitation Type, Heading are number of total.
# Check to see if these columns are correlated with each other
missing_df = df[df['Wet Bulb Temperature'].isnull()]['Station Name'].unique()

print(missing_df == df[df['Total Rain'].isnull()]['Station Name'].unique() == df[df['Precipitation Type'].isnull()]['Station Name'].unique() == df[df['Heading'].isnull()]['Station Name'].unique())

# We can see that all of the columns are missing values only from a single station.

missing_df

# This is the Foster Weather Station.
# Check missing values for Foster Weather Station

foster_df = df[df['Station Name'] == 'Foster Weather Station']

print(foster_df.isnull().sum())

# Compare to non-misisng values from Foster
foster_df.notna().sum()

# We can see that there are zero rows with non-missing values for Foster Station.
# Because the data is entirely missing, we are not able to fill the values with imputation.
# We will drop these rows entirely from the dataset.

# We have already imputated random missing values, so we can safely drop the rest of missing values.
df_cleaned = df.dropna()
len(df_cleaned)



[ True]
Station Name                       0
Measurement Timestamp              0
Air Temperature                    0
Wet Bulb Temperature           75948
Humidity                           0
Rain Intensity                 75948
Interval Rain                      0
Total Rain                     75948
Precipitation Type             75948
Wind Direction                     0
Wind Speed                         0
Maximum Wind Speed                 0
Barometric Pressure                0
Solar Radiation                    0
Heading                        75948
Battery Life                       0
Measurement Timestamp Label        0
Measurement ID                     0
dtype: int64


120367

In [12]:
#Deal with outliers in all numeric columns using iqr
numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns

# looking at the exploration csv, only some columns have noticeable values that could be outliers. For example,
# Solar radiation has a value of -10000000. Because I am not an expert in cutoff values for these categories, 
# I will use IQR method to identify outliers. In columns that look suspect.

numeric_cols_to_check = ['Rain Intensity', 'Interval Rain', 'Barometric Pressure', 'Wind Speed', 'Maximum Wind Speed',
                         'Solar Radiation']

for col in numeric_cols_to_check:
    Q1 = df_cleaned[col].quantile(0.25)
    Q3 = df_cleaned[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)
    df_cleaned.loc[outliers, col] = np.nan

df_cleaned.ffill(inplace = True)

len(df_cleaned)

/var/folders/rv/mml0hwl169g87s6l_19swmnw0000gn/T/ipykernel_37175/474528193.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.ffill(inplace = True)


120367

In [15]:
#Remove duplicates if any
pre_dupes = len(df_cleaned)
df_cleaned = df_cleaned.drop_duplicates()
post_dupes = len(df_cleaned)

In [13]:
#save data

df_cleaned.to_csv('output/q2_data_cleaned.csv', index=False)

In [19]:
df_cleaned.dtypes

#change time column to datetime
df_cleaned['Measurement Timestamp'] = pd.to_datetime(df_cleaned['Measurement Timestamp'])
df_cleaned['Measurement Timestamp Label'] = pd.to_datetime(df_cleaned['Measurement Timestamp Label'])

df_cleaned.dtypes

Station Name                           object
Measurement Timestamp          datetime64[ns]
Air Temperature                       float64
Wet Bulb Temperature                  float64
Humidity                                int64
Rain Intensity                        float64
Interval Rain                         float64
Total Rain                            float64
Precipitation Type                    float64
Wind Direction                          int64
Wind Speed                            float64
Maximum Wind Speed                    float64
Barometric Pressure                   float64
Solar Radiation                       float64
Heading                               float64
Battery Life                          float64
Measurement Timestamp Label    datetime64[ns]
Measurement ID                         object
dtype: object

In [21]:
with open('output/q2_cleaning_report.txt', 'w') as f:
    f.write(f"Rows Before Cleaning: {len(df)}\n\n")
    f.write("Missing Data Description: 7 Columns had missing data. 2 of those columns had random missing data, air temperature and barometric pressure. These were filled using ffill(). The remaining 5 columns, wet bulb temperature, rain intensity, total rain, precipitation type, and heading, had the exact same number of missing data, and all missing values were from Foster Station. Since there was no data from the station at all in these columns, removal was decided rather than attempting to impute data from other stations, as that may not be accurate.\n")
    f.write("\nOutlier Handling: For outliers, the necessary expertise was lacking to set direct boundaries for each of the categories. However, a quick preliminary search allowed for us to determine reasonable minimum and maximums for each column. From this, we only performed outlier transformation on certain columns. We used IQR method to determine outliers. Outliers were set to NA and then filled using ffill(), assuming that outliers were data entry or measurement errors.\n\n")
    f.write(f"Duplicates Removed: {pre_dupes - post_dupes} \n\n")
    f.write(f"Data Types Cleaned: Changed Measurement Timestamp and Measurement Timestamp Label to datetime64.\n\n")
    f.write(f"Rows After Cleaning: {len(df_cleaned)} \n")